In [ ]:
import pandas as pd
import numpy as np
import pickle as p
import time
import requests
import json
from threading import Thread

# Start with this (use a different one for each test)
test_project_id = 'test_6'

url_head = 'https://fillet.azurewebsites.net/'
url_head = 'http://localhost:5000/train/'

data_all = pd.read_csv('../data_all.csv')

# Test with Subset of Data
np.random.seed(43)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=10, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()

# # Test with All Data
# data_subset = data_all.copy()

# <-------------------------- TRAIN --------------------------->

url = url_head+'train/'
data_subset.to_parquet('data_subset.parquet')
files = {'data': open('data_subset.parquet', 'rb')}
payload = {'cv_acc': True, 'project_id': test_project_id, 'modeltype': 'default'}

Thread(target=requests.post,
       args=(url, ),
       kwargs={
           'data': payload,
           'files': files
       }).start()

# give time to startup training
time.sleep(30)
files['data'].close()

# <-------------------------- QUERY PROGRESS --------------------------->


url = url_head+'query_progress/'
payload = {'project_id': test_project_id}
headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=3)

print('% Complete:',round(r.json()['pct_complete'],3))
print('CV % Complete:',round(r.json()['cv_progress'],3))
print('CV Done:',r.json()['cv_done'])


while True:
    try:
        assert round(r.json()['pct_complete'],3) == 100.0
        break
    except:
        continue
    

